# Clickbait Detection: Multimodal Feature Extraction Pipeline

## Overview
Build a comprehensive training dataset for clickbait detection/neutralization that merges:
- **Casual submissions** (labeled as non-clickbait, ~14.7k unique videos)
- **Titles** (356k+ submissions, merged with votes for clickbait scoring)
- **Thumbnails** (~28% of casual videos, with image features)
- **Metadata** (views, category, publish date, upvotes)

### Key Insights
- Text-only: 14.7k pairs from casual submissions
- Multimodal: ~4,100 pairs with both text + image (28% coverage)
- Semi-supervised challenge: 2% of thumbnails labeled → leverage unlabeled pool
- Language distribution: ~97% EN, 1% IT, 0.5% RU, 1.5% other_romance

### Output
A labeled dataset for training:
- Text-only models (LLMs fine-tuning for neutralization)
- Multimodal models (vision-language transformers for clickbait detection)
- Semi-supervised learning on unlabeled submission pool

In [1]:
# Install required libraries (run once)
import subprocess
import sys

packages = [
    'pandas', 'numpy', 'pillow', 'opencv-python', 'pytesseract',
    'langdetect', 'textblob', 'transformers', 'torch', 'torchvision',
    'matplotlib', 'seaborn', 'plotly'
]

# Uncomment to install all at once
# for pkg in packages:
#     subprocess.check_call([sys.executable, '-m', 'pip', 'install', '-q', pkg])

print("✓ Core packages listed. Uncomment above to install all, or use notebook_install_packages tool.")

✓ Core packages listed. Uncomment above to install all, or use notebook_install_packages tool.


In [2]:
# Imports and Setup
from pathlib import Path
import pandas as pd
import numpy as np
import re
from collections import Counter
import json

# NLP and Language Detection
try:
    from langdetect import detect, detect_langs
except ImportError:
    print("langdetect not installed. Install with: pip install langdetect")
    
try:
    from textblob import TextBlob
except ImportError:
    print("textblob not installed. Install with: pip install textblob")

# Image Processing
try:
    from PIL import Image
    import cv2
except ImportError:
    print("PIL/opencv not installed. Install with: pip install pillow opencv-python")

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Data directory
DATA_DIR = Path('../deArrow_data')
print('Data directory:', DATA_DIR.exists(), '\nCurrent working dir:', Path.cwd())

PIL/opencv not installed. Install with: pip install pillow opencv-python
Data directory: True 
Current working dir: c:\Users\Sahar\Desktop\Clickbait_git\Clickbait-project\EDA


In [7]:
# 1. Load Core Data with Merging
def load_and_merge_data():
    # Load casual submissions (labeled as non-clickbait)
    casual_votes = pd.read_csv(DATA_DIR / 'casualVotes.csv')
    casual_titles = pd.read_csv(DATA_DIR / 'casualVoteTitles.csv')
    
    # Load titles with votes (clickbait scoring)
    titles = pd.read_csv(DATA_DIR / 'titles.csv')
    title_votes = pd.read_csv(DATA_DIR / 'titleVotes.csv')
    
    # Load thumbnails (image data reference)
    thumbnails = pd.read_csv(DATA_DIR / 'thumbnails.csv')
    
    # Load video metadata
    video_info = pd.read_csv(DATA_DIR / 'videoInfo.csv')
    
    print("=== Data Loaded ===")
    print(f"casual_votes: {len(casual_votes)} rows, {casual_votes['videoID'].nunique()} unique videos")
    print(f"casual_titles: {len(casual_titles)} rows, {casual_titles['videoID'].nunique()} unique videos")
    print(f"titles: {len(titles)} rows, {titles['videoID'].nunique()} unique videos")
    print(f"thumbnails: {len(thumbnails)} rows, {thumbnails['videoID'].nunique()} unique videos")
    print(f"video_info: {len(video_info)} rows")
    
    # Merge titles with title_votes for vote scores
    titles_with_votes = titles.merge(
        title_votes[['UUID', 'votes']], 
        on='UUID', 
        how='left'
    ).fillna({'votes': 0})
    
    # Identify casual videos: videos with casual_votes (non-clickbait ground truth)
    casual_video_ids = set(casual_votes['videoID'].dropna().unique())
    print(f"\nCasual (non-clickbait) videoIDs: {len(casual_video_ids)}")
    
    # Create base dataset: all videos with titles
    dataset = titles_with_votes[['videoID', 'title', 'UUID', 'votes', 'original', 'casualMode']].copy()
    
    # Add casual label (1 = non-clickbait from casual submissions, 0 = potential clickbait or unlabeled)
    dataset['is_casual'] = dataset['videoID'].isin(casual_video_ids).astype(int)
    
    # Merge with thumbnails (28% coverage)
    dataset = dataset.merge(
        thumbnails[['videoID', 'UUID', 'original', 'timeSubmitted']].rename(
            columns={'UUID': 'thumbnail_UUID', 'original': 'is_original_thumbnail', 'timeSubmitted': 'thumbnail_time'}
        ),
        on='videoID',
        how='left'
    )
    
    # Add video metadata (videoInfo only has videoID, channelID, title, published)
    dataset = dataset.merge(
        video_info[['videoID', 'channelID', 'published']].drop_duplicates(subset=['videoID']),
        on='videoID',
        how='left'
    )
    
    print(f"\nMerged dataset: {len(dataset)} rows")
    print(f"  - With thumbnail_UUID (multimodal): {dataset['thumbnail_UUID'].notna().sum()} ({100*dataset['thumbnail_UUID'].notna().sum()/len(dataset):.1f}%)")
    print(f"  - Labeled casual (non-clickbait): {dataset['is_casual'].sum()}")
    print(f"  - Unlabeled (potential clickbait or semi-supervised): {len(dataset) - dataset['is_casual'].sum()}")
    
    return dataset

dataset = load_and_merge_data()

=== Data Loaded ===
casual_votes: 21590 rows, 14747 unique videos
casual_titles: 12571 rows, 12426 unique videos
titles: 493305 rows, 356427 unique videos
thumbnails: 182406 rows, 162827 unique videos
video_info: 9846585 rows

Casual (non-clickbait) videoIDs: 14747

Merged dataset: 549330 rows
  - With thumbnail_UUID (multimodal): 263569 (48.0%)
  - Labeled casual (non-clickbait): 31053
  - Unlabeled (potential clickbait or semi-supervised): 518277


In [ ]:
# 2. Language Detection Filtering
def detect_language_safe(text):
    """Detect language with fallback to 'unknown'"""
    if pd.isna(text):
        return 'unknown'
    try:
        return detect(str(text)[:500])  # Use first 500 chars for speed
    except:
        return 'unknown'

print("Detecting languages (this may take a minute)...")
dataset['language'] = dataset['title'].apply(detect_language_safe)

lang_dist = dataset['language'].value_counts()
print("\n=== Language Distribution ===")
print(lang_dist)

# Focus on EN, IT, ES for now
focus_langs = {'en', 'it', 'es'}
dataset['language_focus'] = dataset['language'].isin(focus_langs).astype(int)

print(f"\nFocused languages (EN, IT, ES): {dataset['language_focus'].sum()} rows ({100*dataset['language_focus'].sum()/len(dataset):.1f}%)")
print(f"Proceeding with focused language set for feature extraction...")

Detecting languages (this may take a minute)...


In [ ]:
# 3. Text Feature Extraction
def extract_text_features(text):
    """Extract linguistic and sentiment features"""
    if pd.isna(text):
        return {
            'title_length': 0, 'word_count': 0, 'char_per_word': 0,
            'exclamation_count': 0, 'question_count': 0, 'ellipsis_count': 0,
            'uppercase_ratio': 0, 'digit_count': 0, 'emoji_count': 0,
            'caps_words': 0, 'has_caps_sequence': 0,
            'punctuation_density': 0, 'sentiment_polarity': 0, 'sentiment_subjectivity': 0
        }
    
    text_str = str(text).strip()
    words = text_str.split()
    
    # Sentiment analysis
    try:
        blob = TextBlob(text_str)
        sentiment_polarity = blob.sentiment.polarity
        sentiment_subjectivity = blob.sentiment.subjectivity
    except:
        sentiment_polarity = 0
        sentiment_subjectivity = 0
    
    features = {
        'title_length': len(text_str),
        'word_count': len(words),
        'char_per_word': len(text_str) / max(1, len(words)),
        'exclamation_count': text_str.count('!'),
        'question_count': text_str.count('?'),
        'ellipsis_count': text_str.count('...'),
        'uppercase_ratio': sum(1 for c in text_str if c.isupper()) / max(1, len(text_str)),
        'digit_count': sum(1 for c in text_str if c.isdigit()),
        'emoji_count': len(re.findall(r'[😀-🙏🌀-🗿🚀-🛿]', text_str)),  # Simplified emoji detection
        'caps_words': sum(1 for w in words if w.isupper() and len(w) > 1),
        'has_caps_sequence': int(bool(re.search(r'[A-Z]{3,}', text_str))),
        'punctuation_density': sum(1 for c in text_str if c in '!?.,:;-()[]{}') / max(1, len(text_str)),
        'sentiment_polarity': sentiment_polarity,
        'sentiment_subjectivity': sentiment_subjectivity,
    }
    return features

print("Extracting text features...")
text_features = dataset['title'].apply(extract_text_features).apply(pd.Series)
dataset = pd.concat([dataset, text_features.add_prefix('text_')], axis=1)

print(f"✓ Text features added: {[c for c in dataset.columns if c.startswith('text_')][:5]}...")
print(f"\nText feature summary (title length):")
print(dataset['text_title_length'].describe())

In [ ]:
# 4. Image Feature Extraction (Metadata-based for now, actual image download optional)
def extract_image_features_metadata():
    """
    Extract image-related metadata features.
    For actual image processing (brightness, histogram, OCR), load thumbnails via deArrow API.
    Here we create placeholders and document the pipeline.
    """
    
    # Add placeholder columns for image features
    image_features = {
        'image_brightness_mean': np.nan,  # Would compute from PIL.Image
        'image_color_dominant_r': np.nan,  # Color histogram
        'image_color_dominant_g': np.nan,
        'image_color_dominant_b': np.nan,
        'image_face_detected': 0,  # Would use face_recognition library
        'image_text_detected': 0,  # Would use pytesseract/EasyOCR
        'image_text_confidence': 0.0,
    }
    
    for col, val in image_features.items():
        dataset[col] = val
    
    print("✓ Image feature columns created (ready for thumbnail download + processing)")
    print(f"  Columns: {list(image_features.keys())}")
    print(f"\nNote: To extract actual image features:")
    print(f"  1. Download thumbnails from deArrow (videoID → thumbnail URL)")
    print(f"  2. Use PIL for brightness/color histogram")
    print(f"  3. Use face_recognition or mediapipe for face detection")
    print(f"  4. Use pytesseract or EasyOCR for text detection")
    print(f"  5. Integrate CLIP for image-text matching score")

extract_image_features_metadata()

In [ ]:
# 5. Metadata Merging and Labeling
print("=== Metadata Integration ===")

# Clean views and ensure numeric
dataset['views'] = pd.to_numeric(dataset['views'], errors='coerce').fillna(0)

# Create target labels
# Label 0: casual submissions (non-clickbait ground truth)
# Label 1: potential clickbait (high votes → likely clickbait by community)
# Label -1: unlabeled (needs semi-supervised approach)

def create_label(row):
    """
    Labeling strategy:
    - 0: casual submission (ground truth non-clickbait)
    - 1: high-voted original submission (likely clickbait)
    - -1: other (unlabeled, for semi-supervised learning)
    """
    if row['is_casual'] == 1:
        return 0  # Non-clickbait
    elif row['original'] == 1 and row['votes'] > 2:
        return 1  # Clickbait (highly voted original titles)
    else:
        return -1  # Unlabeled

dataset['label'] = dataset.apply(create_label, axis=1)

label_dist = dataset['label'].value_counts().sort_index()
print(f"\nLabel Distribution:")
for lbl in [-1, 0, 1]:
    count = label_dist.get(lbl, 0)
    pct = 100 * count / len(dataset)
    label_name = ['Unlabeled (semi-supervised)', 'Non-clickbait (casual)', 'Clickbait (voted)'][lbl + 1]
    print(f"  {lbl}: {count:>6} ({pct:>5.1f}%) - {label_name}")

# Multimodal coverage
print(f"\n=== Modality Coverage ===")
print(f"Text-only (no thumbnail): {(~dataset['thumbnail_UUID'].notna()).sum()} rows")
print(f"Multimodal (text + image): {dataset['thumbnail_UUID'].notna().sum()} rows")
print(f"  - Non-clickbait (text + image): {dataset[(dataset['label']==0) & (dataset['thumbnail_UUID'].notna())].shape[0]}")
print(f"  - Clickbait (text + image): {dataset[(dataset['label']==1) & (dataset['thumbnail_UUID'].notna())].shape[0]}")

# Show sample
print(f"\n=== Sample Labeled Data ===")
print(dataset[['title', 'is_casual', 'language', 'votes', 'views', 'label']].head(3))

In [ ]:
# 6. Data Balance and Semi-Supervised Planning
print("=== Data Analysis for Model Training ===\n")

# Text-only dataset (labeled)
text_only_labeled = dataset[(dataset['label'] >= 0)]
print(f"TEXT-ONLY DATASET:")
print(f"  Total: {len(text_only_labeled)} rows")
print(f"  Non-clickbait: {(text_only_labeled['label']==0).sum()}")
print(f"  Clickbait: {(text_only_labeled['label']==1).sum()}")
print(f"  Class ratio (non-clickbait:clickbait): {(text_only_labeled['label']==0).sum() / max(1, (text_only_labeled['label']==1).sum()):.2f}:1")

# Multimodal dataset (labeled)
multimodal_labeled = dataset[(dataset['label'] >= 0) & (dataset['thumbnail_UUID'].notna())]
print(f"\nMULTIMODAL DATASET (text + image):")
print(f"  Total: {len(multimodal_labeled)} rows")
print(f"  Non-clickbait: {(multimodal_labeled['label']==0).sum()}")
print(f"  Clickbait: {(multimodal_labeled['label']==1).sum()}")
if (multimodal_labeled['label']==1).sum() > 0:
    print(f"  Class ratio: {(multimodal_labeled['label']==0).sum() / (multimodal_labeled['label']==1).sum():.2f}:1")

# Unlabeled pool (for semi-supervised)
unlabeled = dataset[dataset['label'] == -1]
print(f"\nUNLABELED POOL (semi-supervised learning):")
print(f"  Total: {len(unlabeled)} rows ({100*len(unlabeled)/len(dataset):.1f}%)")
print(f"  Unlabeled with thumbnails: {(unlabeled['thumbnail_UUID'].notna()).sum()}")

# Language breakdown
print(f"\nLANGUAGE BREAKDOWN (focused EN, IT, ES):")
lang_subset = dataset[dataset['language_focus'] == 1]
for lang in ['en', 'it', 'es']:
    lang_count = (lang_subset['language'] == lang).sum()
    print(f"  {lang.upper()}: {lang_count} ({100*lang_count/len(lang_subset):.1f}%)")

# Visualization: Class balance across modalities
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

# Text-only
text_labels = text_only_labeled['label'].value_counts()
axes[0].bar(['Non-CB', 'CB'], [text_labels.get(0, 0), text_labels.get(1, 0)], color=['green', 'red', 'gray'][:2])
axes[0].set_title('Text-Only (Labeled)')
axes[0].set_ylabel('Count')

# Multimodal
if len(multimodal_labeled) > 0:
    mm_labels = multimodal_labeled['label'].value_counts()
    axes[1].bar(['Non-CB', 'CB'], [mm_labels.get(0, 0), mm_labels.get(1, 0)], color=['green', 'red'][:len(mm_labels)])
    axes[1].set_title('Multimodal (Labeled)')
else:
    axes[1].text(0.5, 0.5, 'No multimodal data', ha='center', va='center', transform=axes[1].transAxes)
    axes[1].set_title('Multimodal (Labeled)')

# Unlabeled
axes[2].bar(['Unlabeled'], [len(unlabeled)], color='gray', alpha=0.5)
axes[2].set_title(f'Unlabeled Pool ({100*len(unlabeled)/len(dataset):.0f}%)')
axes[2].set_ylabel('Count')

plt.tight_layout()
plt.savefig('data_balance.png', dpi=100, bbox_inches='tight')
plt.show()

print(f"\n✓ Saved visualization to data_balance.png")

In [ ]:
# 7. Save Labeled Datasets and Feature Matrix
import json

# Select feature columns for output
feature_cols = [c for c in dataset.columns if c.startswith('text_')]
output_cols = [
    'videoID', 'title', 'language', 'label', 'is_casual', 'votes', 'views',
    'thumbnail_UUID', 'UUID'
] + feature_cols

# Text-only (labeled)
text_only_labeled_out = text_only_labeled[output_cols].copy()
text_only_labeled_out.to_csv('training_data_text_only.csv', index=False)
print(f"✓ Saved text-only training data: {len(text_only_labeled_out)} rows to training_data_text_only.csv")

# Multimodal (labeled)
if len(multimodal_labeled) > 0:
    multimodal_out = multimodal_labeled[output_cols].copy()
    multimodal_out.to_csv('training_data_multimodal.csv', index=False)
    print(f"✓ Saved multimodal training data: {len(multimodal_out)} rows to training_data_multimodal.csv")

# Unlabeled pool (for semi-supervised)
unlabeled_out = unlabeled[output_cols].copy()
unlabeled_out.to_csv('unlabeled_pool.csv', index=False)
print(f"✓ Saved unlabeled pool: {len(unlabeled_out)} rows to unlabeled_pool.csv")

# Full feature matrix
dataset_out = dataset[output_cols].copy()
dataset_out.to_csv('training_data_full.csv', index=False)
print(f"✓ Saved full feature matrix: {len(dataset_out)} rows to training_data_full.csv")

# Save summary metadata
metadata = {
    'dataset_name': 'deArrow Clickbait Detection',
    'total_samples': len(dataset),
    'text_only_labeled': len(text_only_labeled),
    'multimodal_labeled': len(multimodal_labeled),
    'unlabeled': len(unlabeled),
    'language_focus': ['en', 'it', 'es'],
    'text_features': feature_cols,
    'image_features_placeholder': [
        'image_brightness_mean', 'image_color_dominant_r', 'image_color_dominant_g',
        'image_color_dominant_b', 'image_face_detected', 'image_text_detected'
    ],
    'labels': {
        '0': 'Non-clickbait (casual submissions)',
        '1': 'Clickbait (voted originals)',
        '-1': 'Unlabeled (semi-supervised)'
    }
}

with open('training_metadata.json', 'w') as f:
    json.dump(metadata, f, indent=2)

print(f"\n✓ Saved metadata to training_metadata.json")
print(f"\nDataset Summary:")
print(json.dumps(metadata, indent=2))

In [ ]:
# 8. Model Training Recommendations
print("=" * 80)
print("MODEL TRAINING STRATEGIES")
print("=" * 80)

recommendations = """
### 1. TEXT-ONLY MODELS (14.7k+ samples)
   **Architecture:** Fine-tune pre-trained LLM on clickbait neutralization
   
   **Datasets:**
   - Supervised: ~14.7k casual (non-clickbait) vs. ~7k voted originals
   - Semi-supervised: 350k+ unlabeled submissions
   
   **Recommended Approaches:**
   a) Fine-tune on task: Seq2Seq (T5, BART) for clickbait → neutral generation
   b) Classification: RoBERTa, DistilBERT for binary clickbait detection
   c) Semi-supervised: Pseudo-labeling on unlabeled pool using uncertainty sampling
   
   **Libraries:** transformers, torch, pytorch-lightning

### 2. MULTIMODAL MODELS (4.1k+ samples with images)
   **Architecture:** Vision-Language transformers (CLIP, ViLBERT, LXMERT)
   
   **Datasets:**
   - Text + image pairs: ~4.1k labeled (only ~28% coverage)
   - Image-only features: brightness, color histogram, face/text detection
   
   **Recommended Approaches:**
   a) Joint embedding: Use CLIP to learn shared text-image space
   b) Classification: Fine-tune ViLBERT for multimodal clickbait detection
   c) Data augmentation: Generate synthetic negatives from unlabeled images
   
   **Libraries:** transformers, timm, torchvision, CLIP
   
### 3. SEMI-SUPERVISED STRATEGY (350k unlabeled)
   **Challenge:** Only 2% of thumbnails labeled → exploit unlabeled pool
   
   **Approaches:**
   a) Pseudo-labeling: Use confident model predictions to label unlabeled data
   b) Self-training: Iteratively train on pseudo-labeled pool + real labels
   c) Contrastive learning: Learn representations from unlabeled pool (SimCLR)
   d) Consistency regularization: MixMatch, FixMatch for image/text
   
   **Libraries:** semi-supervised-learning libraries, transformers

### 4. HANDLING CLASS IMBALANCE
   **Issue:** Non-clickbait heavily outweighs clickbait in labeled set
   
   **Solutions:**
   a) Weighted loss: BCE with class weights
   b) Focal loss: Focus on hard negative examples
   c) Oversampling: SMOTE for synthetic clickbait examples
   d) Threshold tuning: Adjust decision boundary for F1/precision
   
   **Libraries:** scikit-learn, torch, imbalanced-learn

### 5. FEATURE ENGINEERING ENHANCEMENTS
   **Image features to extract:**
   - Brightness, saturation, contrast (PIL/cv2)
   - Dominant colors, color entropy (histogram)
   - Face detection + count (face_recognition, mediapipe)
   - Text detection + OCR (pytesseract, EasyOCR)
   - Object detection (YOLOv5) for sensational imagery
   
   **Libraries:** opencv-python, PIL, face_recognition, pytesseract, yolov5

### 6. EVALUATION METRICS
   - Precision, Recall, F1 (imbalanced data)
   - AUC-ROC (ranking metric)
   - Human evaluation on test set (for neutralization task)
   - Language-specific performance (EN vs. IT vs. ES)

### 7. TRAINING PIPELINE
   1. Start with text-only models (faster feedback)
   2. Validate on multimodal subset (4.1k samples)
   3. Use unlabeled pool for semi-supervised fine-tuning
   4. Deploy ensemble (text + multimodal) for robust predictions
"""

print(recommendations)

# Save recommendations
with open('training_recommendations.txt', 'w') as f:
    f.write(recommendations)

print("✓ Saved recommendations to training_recommendations.txt")